In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer, OrdinalEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.decomposition import PCA
#from category_encoders import TargetEncoder
from pandas.api.types import CategoricalDtype

In [ ]:
# Leer el JSON
df_91_20 = pd.read_json('df_91_20.json', orient='records', lines=True)
df_91_20.head()

---
## Variable Target

Para abordar el problema como una tarea de clasificación, se definió como variable objetivo la temperatura media anual de cada estación, representada por la variable 'Temperatura'. A partir de su valor promedio anual, se clasificaron las estaciones meteorológicas en cuatro categorías:

Frías: promedio menor a 10 °C

Templadas frescas: entre 10 °C y 15 °C

Templadas cálidas: entre 15 °C y 20 °C

Cálidas: promedio mayor a 20 °C

Esta clasificación busca representar de manera más precisa la variabilidad climática del país. A continuación, se agruparon las observaciones por estación y se asignó una etiqueta a cada una según el promedio de temperatura.

In [ ]:
# Agrupamos por estación y calculamos el promedio de temperatura

df_temperatura_estacion = df_91_20.groupby("Estación")["Temperatura"].mean().reset_index()

# Creamos la variable target

def clasificar_temp(temperatura):
    if temperatura < 10:
        return "fría"
    elif temperatura < 15:
        return "templada fresca"
    elif temperatura < 20:
        return "templada cálida"
    else:
        return "cálida"

df_temperatura_estacion["CLASE"] = df_temperatura_estacion["Temperatura"].apply(clasificar_temp)

In [ ]:
print(df_temperatura_estacion["CLASE"].value_counts())

---
## Desbalance de clases

Se nota un fuerte desbalance de clases siendo la clase "templada cálida" fuertemente predominante. Este desbalance puede afectar negativamente la capacidad del modelo para aprender patrones representativos de las clases minoritarias y esto puede llevar a modelos sesgados o con bajo poder predictivo. Graficamos para mayor claridad: 

In [ ]:
sns.countplot(data=df_temperatura_estacion, x="CLASE", palette="coolwarm")
plt.title("Distribución de clases por estación")
plt.xlabel("Clase térmica")
plt.ylabel("Cantidad de estaciones")
plt.show()

Para corregir este sobremuestreo elegimos aplicar sobremuestreo con SMOTE a las clases minoritarias en lugar de un submuestreo de la clase mayoritaria ya que nos permite conservar mayor cantidad de datos y ayuda a evitar el overfitting. 

Primero imputamos los valores faltantes y transformamos las variables categóricas en los pipelines.

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Definimos variables numéricas y categóricas
numericas = df_temperatura_estacion.select_dtypes(include='number').drop(columns=["Temperatura"]).columns.tolist()
categoricas = df_temperatura_estacion.select_dtypes(include='object').drop(columns=["CLASE"]).columns.tolist()

# Pipelines para imputación y encoding
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer para las variables numéricas y categóricas
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numericas),
    ("cat", categorical_pipeline, categoricas)
])

# Dividimos el dataset antes del preprocesamiento para evitar el data leakage
X = df_temperatura_estacion.drop(columns=["CLASE"])
y = df_temperatura_estacion["CLASE"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Ajustamos sólo con el set de entrenamiento
preprocessor.fit(X_train)

#Transformamos
X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)

# SMOTE sobre el set de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_prep, y_train)

print(y_train_res.value_counts())

CLASE
templada cálida    40
fría               40
cálida             40
templada fresca    40
Name: count, dtype: int64


Luego de aplicar SMOTE sobre el set de entrenamiento vemos que ahora se logró una distribución balanceada de clases (40 estaciones cada una), esto proporciona una buena base para el posterior entrenamiento del modelo. Nótese que en lugar de obtener 50 estaciones por clase, al haber hecho la división de 80/20 antes de SMOTE la clase mayoritaria pasó a tener 40 ítems, eso explica este número para las demás clases.

 

--- 
## Análisis y selección de features



